<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#模組化回測系統" data-toc-modified-id="模組化回測系統-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>模組化回測系統</a></span></li><li><span><a href="#模組化" data-toc-modified-id="模組化-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>模組化</a></span></li><li><span><a href="#選擇兩檔股票" data-toc-modified-id="選擇兩檔股票-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>選擇兩檔股票</a></span></li><li><span><a href="#檢視大盤績效" data-toc-modified-id="檢視大盤績效-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>檢視大盤績效</a></span></li><li><span><a href="#檢視MSCI投組" data-toc-modified-id="檢視MSCI投組-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>檢視MSCI投組</a></span></li></ul></div>

# 模組化回測系統

In [30]:
import numpy as np
import pandas as pd

In [31]:
import tejapi
tejapi.ApiConfig.api_key = 'Your Key'
tejapi.ApiConfig.ignoretz = True

交易策略
- 標的：大盤、MSCI指數成分股
- 價量策略：追漲殺跌。股價、成交量突破近20日最高價格(量)時買入，向下突破近10日最低價格賣出
- 持有策略：持有至最終日

# 模組化

In [32]:
class backtest():
    
    def __init__(self, target_list):
        # 設定初始值
        
        # 股票池
        self.target_list = target_list

        # 訊號表
        self.signal_data = pd.DataFrame()
        
        # 交易表
        self.action_data = pd.DataFrame()
        
        # 庫存表
        self.position = pd.DataFrame()
        
        # 獲利數
        self.protfolio_profit = []
        
        # 成本數
        self.protfolio_cost = []
        
        # 資料庫
        self.data = tejapi.get('TWN/APRCD',
                               coid = target_list,
                               mdate={'gte':'2020-01-01', 'lte':'2020-12-31'},
                               opts={'columns':['coid','mdate','close_d', 'volume']},
                               chinese_column_name=True,
                               paginate=True).reset_index(drop=True)
        

    def price_volume(self, specific=None):
        # 策略更新
        self.signal_data = pd.DataFrame()
        self.action_data = pd.DataFrame()
        
        # 股票池每一檔股票都跑一次訊號產生
        for i in self.target_list:
            target_data = self.data[self.data['證券代碼'] == i]
            rolling_max = target_data[['收盤價(元)', '成交量(千股)']].rolling(20).max()
            rolling_max.columns=['收盤價(max)','成交量(max)'] 

            rolling_min = target_data[['收盤價(元)', '成交量(千股)']].rolling(10).min()
            rolling_min.columns=['收盤價(min)','成交量(min)']
            
            merge_data = pd.concat([rolling_max, rolling_min], axis=1)
            stock_data = pd.concat([target_data, merge_data], axis=1)
            
            #收盤價、成交量同時突破近20天高點買入
            stock_data['買入訊號判斷'] = np.where((stock_data['收盤價(元)'] == stock_data['收盤價(max)']) & (stock_data['成交量(千股)'] == stock_data['成交量(max)']), -1, 0)

            #收盤價、成交量同時突破近20天低點賣入
            stock_data['賣出訊號判斷'] = np.where((stock_data['收盤價(元)'] == stock_data['收盤價(min)']) & (stock_data['成交量(千股)'] == stock_data['成交量(min)']), 1, 0)

            self.signal_data = pd.concat([self.signal_data, stock_data], axis=0)
            
            # 最後一天平倉
            remain_stock = self.signal_data.iloc[:-1,8:10].sum().sum()
            self.signal_data.iloc[-1,9] = 0
            self.signal_data.iloc[-1,8] = 0
            if remain_stock < 0:
                self.signal_data.iloc[-1,9] = -remain_stock
            else:
                self.signal_data.iloc[-1,8] = -remain_stock
            
        # 如果沒選定特定股票，則會顯示全部
        if specific == None:
            specific = self.target_list

        return self.signal_data.loc[self.signal_data['證券代碼'].isin(specific)]
    
    
    def buy_hold(self, specific=None):
        # 策略更新
        self.signal_data = pd.DataFrame()
        self.action_data = pd.DataFrame()
        
        for i in self.target_list:
            target_data = self.data[self.data['證券代碼'] == i].copy()
            
            target_data['買入訊號判斷'] = 0
            target_data['賣出訊號判斷'] = 0
            
            # 第一天買入，最後一天賣出
            target_data.iloc[0, 4] = -1
            target_data.iloc[-1,5] = 1

            self.signal_data = pd.concat([self.signal_data, target_data], axis=0)

        # 如果沒選定特定股票，則會顯示全部
        if specific == None:
            specific = self.target_list
            
        return self.signal_data.loc[self.signal_data['證券代碼'].isin(specific)]
        
    
    def run(self, specific=None):
        # 做出交易紀錄表
        trade_data = pd.DataFrame(index= self.data['年月日'], columns=self.target_list).fillna(0.0)
        
        action_data = self.signal_data[(self.signal_data['買入訊號判斷'] != 0) | (self.signal_data['賣出訊號判斷'] != 0)]
        
        action_data = action_data[['證券代碼','年月日','收盤價(元)','買入訊號判斷', '賣出訊號判斷']].sort_values(by = '年月日').reset_index(drop=True) 
        
        action_data['交易'] = action_data['收盤價(元)'] * (action_data['買入訊號判斷'] + action_data['賣出訊號判斷'])
        
        # 計算個股總獲利
        self.protfolio_profit.append(sum(action_data['交易'].tolist()))
        
        # 計算庫存
        action_data['持有股數'] = (action_data['買入訊號判斷']+action_data['賣出訊號判斷']).cumsum()
        action_data['資金'] = action_data['交易'].cumsum()
        
        # 計算個股總成本，買入是負數，所以前面加上負號
        self.protfolio_cost.append(-action_data[action_data['買入訊號判斷'] < 0]['交易'].sum())
        
        self.action_data = action_data
        
        # 如果沒選定特定股票，則會顯示全部
        if specific == None:
            specific = self.target_list

        return self.action_data.loc[self.action_data['證券代碼'].isin(specific)]
    
    
    def ROI(self):
        # 報酬率
        return_profit = sum(self.protfolio_profit) / sum(self.protfolio_cost)
        
        return return_profit
    


# 選擇兩檔股票

In [33]:
# 先選定股票池
bt = backtest(['2330','1303'])

In [34]:
# 選擇策略
bt.price_volume()

,證券代碼,年月日,收盤價(元),成交量(千股),收盤價(max),成交量(max),收盤價(min),成交量(min),買入訊號判斷,賣出訊號判斷
245,2330,2020-01-02,339.0,33282.120,NaN,NaN,NaN,NaN,0,0
246,2330,2020-01-03,339.5,42023.268,NaN,NaN,NaN,NaN,0,0
247,2330,2020-01-06,332.0,45677.057,NaN,NaN,NaN,NaN,0,0
248,2330,2020-01-07,329.5,51746.181,NaN,NaN,NaN,NaN,0,0
249,2330,2020-01-08,329.5,37913.748,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...
240,1303,2020-12-25,69.4,3134.360,69.4,22411.990,67.2,2722.715,0,0
241,1303,2020-12-28,70.6,6930.374,70.6,17457.807,67.2,2722.715,0,0
242,1303,2020-12-29,70.4,3126.134,70.6,17457.807,67.5,2722.715,0,0
243,1303,2020-12-30,72.5,12444.962,72.5,17457.807,67.5,2722.715,0,0


In [35]:
# 執行回測
bt.run()

,證券代碼,年月日,收盤價(元),買入訊號判斷,賣出訊號判斷,交易,持有股數,資金
0,1303,2020-04-29,62.5,-1,0,-62.5,-1,-62.5
1,1303,2020-04-30,66.2,-1,0,-66.2,-2,-128.7
2,2330,2020-07-06,338.0,-1,0,-338.0,-3,-466.7
3,2330,2020-07-07,338.5,-1,0,-338.5,-4,-805.2
4,2330,2020-07-10,348.5,-1,0,-348.5,-5,-1153.7
5,2330,2020-07-14,363.5,-1,0,-363.5,-6,-1517.2
6,2330,2020-07-24,386.0,-1,0,-386.0,-7,-1903.2
7,2330,2020-07-27,424.5,-1,0,-424.5,-8,-2327.7
8,2330,2020-07-28,435.0,-1,0,-435.0,-9,-2762.7
9,1303,2020-07-29,61.1,0,1,61.1,-8,-2701.6


In [36]:
# 查看報酬
bt.ROI()

0.34690608925986394

In [37]:
# 檢視總收益
bt.protfolio_profit

[1126.3000000000002]

In [38]:
# 檢視總成本
bt.protfolio_cost

[3246.7]

# 檢視大盤績效

In [39]:
market = backtest(['Y9997'])

In [40]:
market.buy_hold()

,證券代碼,年月日,收盤價(元),成交量(千股),買入訊號判斷,賣出訊號判斷
0,Y9997,2020-01-02,22566.08,4335855.0,-1,0
1,Y9997,2020-01-03,22584.65,5527302.0,0,0
2,Y9997,2020-01-06,22291.72,4679171.0,0,0
3,Y9997,2020-01-07,22155.51,4659753.0,0,0
4,Y9997,2020-01-08,22037.60,5076957.0,0,0
...,...,...,...,...,...,...
240,Y9997,2020-12-25,27667.00,7354213.0,0,0
241,Y9997,2020-12-28,27959.77,8522466.0,0,0
242,Y9997,2020-12-29,27938.50,9485864.0,0,0
243,Y9997,2020-12-30,28354.81,8345169.0,0,0


In [41]:
market.run()

,證券代碼,年月日,收盤價(元),買入訊號判斷,賣出訊號判斷,交易,持有股數,資金
0,Y9997,2020-01-02,22566.08,-1,0,-22566.08,-1,-22566.08
1,Y9997,2020-12-31,28441.36,0,1,28441.36,0,5875.28


In [42]:
market.ROI()

0.26035891036458253

In [43]:
market.price_volume()

,證券代碼,年月日,收盤價(元),成交量(千股),收盤價(max),成交量(max),收盤價(min),成交量(min),買入訊號判斷,賣出訊號判斷
0,Y9997,2020-01-02,22566.08,4335855.0,NaN,NaN,NaN,NaN,0,0
1,Y9997,2020-01-03,22584.65,5527302.0,NaN,NaN,NaN,NaN,0,0
2,Y9997,2020-01-06,22291.72,4679171.0,NaN,NaN,NaN,NaN,0,0
3,Y9997,2020-01-07,22155.51,4659753.0,NaN,NaN,NaN,NaN,0,0
4,Y9997,2020-01-08,22037.60,5076957.0,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...
240,Y9997,2020-12-25,27667.00,7354213.0,27770.36,10869811.0,27118.44,6227173.0,0,0
241,Y9997,2020-12-28,27959.77,8522466.0,27959.77,10869811.0,27118.44,6227173.0,0,0
242,Y9997,2020-12-29,27938.50,9485864.0,27959.77,10869811.0,27369.78,6227173.0,0,0
243,Y9997,2020-12-30,28354.81,8345169.0,28354.81,10869811.0,27369.78,6227173.0,0,0


In [44]:
market.run()

,證券代碼,年月日,收盤價(元),買入訊號判斷,賣出訊號判斷,交易,持有股數,資金
0,Y9997,2020-04-30,20555.45,-1,0,-20555.45,-1,-20555.45
1,Y9997,2020-06-03,21171.10,-1,0,-21171.10,-2,-41726.55
2,Y9997,2020-07-06,22916.04,-1,0,-22916.04,-3,-64642.59
3,Y9997,2020-09-23,24253.82,0,1,24253.82,-2,-40388.77
4,Y9997,2020-11-09,25303.93,-1,0,-25303.93,-3,-65692.70
5,Y9997,2020-11-11,25563.60,-1,0,-25563.60,-4,-91256.30
6,Y9997,2020-12-07,27480.95,-1,0,-27480.95,-5,-118737.25
7,Y9997,2020-12-31,28441.36,0,5,142206.80,0,23469.55


In [45]:
market.ROI()

0.1772489439447343

In [46]:
# 持有策略有5875的收益，價量有23469收益，但報酬率是持有策略較高
market.protfolio_profit

[5875.279999999999, 23469.549999999974]

# 檢視MSCI投組

In [47]:
data_MSCI = tejapi.get('TWN/AIDXS',
                       coid = 'MSCI',
                       mdate= '2019-12-31',
                       opts={'columns':['coid','mdate','key3']},
                       chinese_column_name=True)

MSCI_list = data_MSCI['成份股'].unique().tolist()
MSCI_list = [coid[:4] for coid in MSCI_list]

In [55]:
MSCI_list

['1101',
 '1102',
 '1216',
 '1227',
 '1301',
 '1303',
 '1326',
 '1402',
 '1434',
 '1476',
 '1590',
 '2002',
 '2049',
 '2105',
 '2207',
 '2301',
 '2303',
 '2308',
 '2317',
 '2324',
 '2327',
 '2330',
 '2344',
 '2345',
 '2347',
 '2353',
 '2354',
 '2356',
 '2357',
 '2371',
 '2377',
 '2379',
 '2382',
 '2385',
 '2395',
 '2408',
 '2409',
 '2412',
 '2454',
 '2474',
 '2492',
 '2542',
 '2603',
 '2610',
 '2618',
 '2633',
 '2801',
 '2823',
 '2834',
 '2880',
 '2881',
 '2882',
 '2883',
 '2884',
 '2885',
 '2886',
 '2887',
 '2888',
 '2890',
 '2891',
 '2892',
 '2912',
 '2915',
 '3008',
 '3034',
 '3045',
 '3105',
 '3231',
 '3481',
 '3702',
 '3711',
 '4904',
 '4938',
 '4958',
 '5347',
 '5871',
 '5876',
 '5880',
 '6239',
 '6488',
 '6505',
 '6669',
 '8299',
 '8464',
 '9904',
 '9910',
 '9921',
 '9945']

In [48]:
# 先選定股票池
msci = backtest(MSCI_list)

In [49]:
# 選擇策略
msci.price_volume()

,證券代碼,年月日,收盤價(元),成交量(千股),收盤價(max),成交量(max),收盤價(min),成交量(min),買入訊號判斷,賣出訊號判斷
0,1101,2020-01-02,44.10,18470.566,NaN,NaN,NaN,NaN,0,0
1,1101,2020-01-03,43.95,18387.114,NaN,NaN,NaN,NaN,0,0
2,1101,2020-01-06,43.45,13867.625,NaN,NaN,NaN,NaN,0,0
3,1101,2020-01-07,43.60,14185.347,NaN,NaN,NaN,NaN,0,0
4,1101,2020-01-08,43.40,13465.670,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...
21555,9945,2020-12-25,40.60,2166.527,42.05,18190.691,40.1,2166.527,0,0
21556,9945,2020-12-28,40.55,3042.628,41.75,15115.806,40.1,2166.527,0,0
21557,9945,2020-12-29,40.60,3399.424,41.75,15115.806,40.1,2166.527,0,0
21558,9945,2020-12-30,40.75,4281.605,41.75,15115.806,40.1,2166.527,0,0


In [50]:
# 進行回測
msci.run()

,證券代碼,年月日,收盤價(元),買入訊號判斷,賣出訊號判斷,交易,持有股數,資金
0,9921,2020-01-16,193.00,0,1,193.0,1,193.00
1,6505,2020-01-16,95.70,0,1,95.7,2,288.70
2,2412,2020-02-05,108.00,0,1,108.0,3,396.70
3,2474,2020-02-07,236.00,0,1,236.0,4,632.70
4,2474,2020-02-10,231.00,0,1,231.0,5,863.70
...,...,...,...,...,...,...,...,...
689,2610,2020-12-31,12.05,0,4,48.2,-21,4105.97
690,5347,2020-12-31,116.00,0,8,928.0,-13,5033.97
691,2618,2020-12-31,13.15,0,2,26.3,-11,5060.27
692,6505,2020-12-31,99.80,0,5,499.0,-6,5559.27


In [51]:
# 檢視報酬率
msci.ROI()

0.08795691517789303

In [52]:
msci.buy_hold()

,證券代碼,年月日,收盤價(元),成交量(千股),買入訊號判斷,賣出訊號判斷
0,1101,2020-01-02,44.10,18470.566,-1,0
1,1101,2020-01-03,43.95,18387.114,0,0
2,1101,2020-01-06,43.45,13867.625,0,0
3,1101,2020-01-07,43.60,14185.347,0,0
4,1101,2020-01-08,43.40,13465.670,0,0
...,...,...,...,...,...,...
21555,9945,2020-12-25,40.60,2166.527,0,0
21556,9945,2020-12-28,40.55,3042.628,0,0
21557,9945,2020-12-29,40.60,3399.424,0,0
21558,9945,2020-12-30,40.75,4281.605,0,0


In [53]:
msci.run()

,證券代碼,年月日,收盤價(元),買入訊號判斷,賣出訊號判斷,交易,持有股數,資金
0,1101,2020-01-02,44.10,-1,0,-44.10,-1,-44.10
1,2492,2020-01-02,239.50,-1,0,-239.50,-2,-283.60
2,3231,2020-01-02,28.40,-1,0,-28.40,-3,-312.00
3,2474,2020-01-02,232.00,-1,0,-232.00,-4,-544.00
4,2454,2020-01-02,441.50,-1,0,-441.50,-5,-985.50
...,...,...,...,...,...,...,...,...
171,2379,2020-12-31,390.50,0,1,390.50,-4,151.46
172,2377,2020-12-31,132.50,0,1,132.50,-3,283.96
173,2371,2020-12-31,26.45,0,1,26.45,-2,310.41
174,2542,2020-12-31,45.85,0,1,45.85,-1,356.26


In [54]:
msci.ROI()

0.07596269617395723